In [1]:
%reset
import os
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import ta

RawStockDataDirectory = 'Data/Raw/Stock/'
ProcessedStockDataDirectory = 'Data/Processed/'

In [2]:
allData = list()

for stockData in os.listdir(RawStockDataDirectory):
    allData.append(pd.read_csv(RawStockDataDirectory + stockData, usecols=[1,2,3,4,5]))

In [3]:
allData[0]

,Open,High,Low,Close,Volume
0,0.000000,0.453585,0.444327,0.444327,14728800
1,0.444327,0.453585,0.444327,0.444327,6511200
2,0.462841,0.472098,0.462841,0.462841,2380800
3,0.462841,0.472098,0.462841,0.462841,1317600
4,0.472098,0.481355,0.472098,0.472098,1128000
...,...,...,...,...,...
9496,208.000000,211.119995,206.970001,208.330002,1299100
9497,209.000000,211.509995,205.070007,205.479996,1209800
9498,205.839996,207.580002,203.070007,204.509995,1012300
9499,205.710007,208.750000,199.899994,200.490005,1439700


In [4]:
count = 1
for stock in allData:
    # Calculate moving averages
    stock['MA10'] = stock['Close'].rolling(window=10).mean()
    stock['MA20'] = stock['Close'].rolling(window=20).mean()
    stock['MA50'] = stock['Close'].rolling(window=50).mean()
    stock['MA100'] = stock['Close'].rolling(window=100).mean()
    stock['MA200'] = stock['Close'].rolling(window=200).mean()

    # Calculate RSI
    stock['RSI'] = ta.momentum.rsi(stock['Close'], window=14)

    # Calculate Bollinger Bands
    BB = ta.volatility.BollingerBands(stock['Close'])
    stock['BBup'], stock['BBmid'], stock['BBlow'] = BB.bollinger_hband(), BB.bollinger_mavg(), BB.bollinger_lband()

    # Calculate MACD
    MACD = ta.trend.MACD(stock['Close'])
    stock['MACD'], stock['MACDsig'], stock['MACDdiff'] = MACD.macd(), MACD.macd_signal(), MACD.macd_diff()

    # Calculate ROC
    stock['ROC'] = ta.momentum.ROCIndicator(stock['Close']).roc()

    # Calculate KAMA
    stock['KAMA'] = ta.momentum.kama(stock['Close'])

    # Calculate OBV
    stock['OBX'] = ta.volume.OnBalanceVolumeIndicator(close=stock['Close'], volume=stock['Volume']).on_balance_volume()

    # Calculate STOCH
    STOCH = ta.momentum.StochasticOscillator(close=stock['Close'], high=stock['High'], low=stock['Low'])
    stock['STOCH'] = STOCH.stoch()
    stock['STOCHsig'] = STOCH.stoch_signal()

    stock.dropna(inplace=True)
    stock.reset_index(drop=True, inplace=True)

    stock.to_csv(ProcessedStockDataDirectory + "Processed_" + str(count))
    count += 1

In [5]:
allData[0]

,Open,High,Low,Close,Volume,MA10,MA20,MA50,MA100,MA200,...,BBmid,BBlow,MACD,MACDsig,MACDdiff,ROC,KAMA,OBX,STOCH,STOCHsig
0,0.971966,0.971966,0.962709,0.971966,93600,0.934939,0.953916,0.985944,0.915962,0.731891,...,0.953916,0.878207,-0.008528,-0.014172,0.005644,1.941681,0.919770,101008800,99.999500,91.110638
1,0.971966,1.018250,0.971966,1.018250,691200,0.945121,0.954378,0.987981,0.919573,0.734760,...,0.954378,0.877229,-0.002573,-0.011852,0.009279,8.910886,0.936578,101700000,100.000000,99.999666
2,1.018250,1.055278,1.018250,1.032135,324000,0.956692,0.955535,0.990295,0.923044,0.737699,...,0.955535,0.874546,0.003229,-0.008836,0.012065,12.060246,0.954385,102024000,88.095086,96.031529
3,1.032135,1.133960,1.032135,1.101561,2656800,0.977057,0.959701,0.993997,0.926839,0.740893,...,0.959701,0.859862,0.013276,-0.004414,0.017690,20.201885,1.002880,104680800,88.135375,92.076820
4,1.101561,1.110818,1.101561,1.101561,456000,0.999273,0.965023,0.997700,0.930634,0.744086,...,0.965023,0.848282,0.020997,0.000668,0.020328,20.201885,1.046738,105136800,88.135375,88.121945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9297,208.000000,211.119995,206.970001,208.330002,1299100,205.461002,214.440000,205.356000,204.102400,200.866400,...,214.440000,191.670252,-2.135909,-0.147563,-1.988345,-9.137298,206.950883,1393633300,44.576404,37.476913
9298,209.000000,211.509995,205.070007,205.479996,1209800,204.454001,213.797000,205.621000,204.246900,200.953900,...,213.797000,190.779138,-2.146318,-0.547314,-1.599004,-7.495615,206.908398,1392423500,37.064391,41.307031
9299,205.839996,207.580002,203.070007,204.509995,1012300,203.235001,212.840000,205.949200,204.355600,201.007850,...,212.840000,189.949129,-2.207394,-0.879330,-1.328063,-7.032458,206.819476,1391411200,38.497771,40.046189
9300,205.710007,208.750000,199.899994,200.490005,1439700,201.168001,211.798500,206.195800,204.422000,201.053250,...,211.798500,188.651676,-2.550772,-1.213619,-1.337153,-6.986777,206.311490,1389971500,30.056469,35.206211
